# 자연어 처리(요약)

In [ ]:
import pandas as pd
import nltk
import numpy as np
import wikipedia

import requests
from bs4 import BeautifulSoup  
from urllib.request import urlopen

import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import datetime
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize


In [ ]:
#nltk.download()

### 데이터 크롤링

In [ ]:
#셀리옴으로 크롤링하기

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
options.add_argument("--disable-gpu")

In [ ]:
driver = webdriver.Chrome('C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\selenium\\webdriver\\chrome\\chromedriver.exe', chrome_options=options)
time.sleep(3)

In [ ]:
#나라별 도시 크롤링
country =  ["스페인", "포르투갈"]
dic = {}
for idx in country :
    dic[idx] = []

In [ ]:
tot = 0
for nature in country :
    base_url = 'https://www.google.co.kr/search?' +'q=' + nature + '+도시&hl=en'
    driver.get(base_url)
    time.sleep(3)
    
    lst = driver.find_elements_by_css_selector("a.klitem")
    for city in lst :
        dic[nature].append(city.text.split("\n")[0])
        tot += 1
        
print(tot)

In [ ]:
result = []
err_lst = []

regex1 = ">(.*)<"
regex2 = "Rated (.*) out"
regex3 = "\((.*)\)"
regex4 = ">(.*)<"
regex5 = ""

tot = 1
a = 0
total = 0
for nature in country :
    for city in dic[nature] :
        print(tot, nature, city, end = " ")
        a+=1
        
        try : 
            #hl=en 을 넣어주면 영어 검색
            base_url = 'https://www.google.co.kr/search?' +'q=' + nature + '+' + city + '+' + "관광지&hl=en"
            driver.get(base_url)
            time.sleep(5)
            
            #mor_buttin_id가 수시로 바뀌니 체크하기
            more_button_id = '_x7n'
            driver.find_element_by_class_name(more_button_id).click()
            time.sleep(5)

            html =  driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            temp = soup.select("body > div > div > div > ol > li > div > div")
            
            total = 0
            for idx, place in enumerate(temp) :
                soup = BeautifulSoup(str(place), "html.parser")
                
                name = re.search(regex1, str(soup.select("div > h2")[0])).group(1)
                
                try : star = float(re.search(regex2, str(soup.select("div >  g-review-stars > span")[0])).group(1))
                except : star = 0
                    
                try : review = int(re.search(regex3, str(soup.select("div > div  > span._Mnc")[0])).group(1).replace(",", ""))
                except : review = 0
                
                try : short_info = re.search(regex4, str(soup.select("p._DAm")[0])).group(1)
                except : short_info = ""
                    
                try : long_info = str(soup.select("p._BAm")[0]).split(">")[1].split("<")[0]
                except : long_info = ""
                
                #위키피디아 검색
                
                try :
                    page = wikipedia.page(name)
                    time.sleep(5)
                    summary = page.summary
                    content = page.content
                    time.sleep(5)
                except :
                    summary = ""
                    content = "" 
                
                result.append((name, (nature, city, star, review, short_info, long_info, content, summary)))
                total += 1
                    
                
            print("***",total, "clear")
        except :
            err_lst.append((nature, city))
            print("   ","err_fin")
        

        tot+=1
    
    

        

In [ ]:
result[0]

In [ ]:
wiki_df = pd.DataFrame.from_items(result).T
wiki_df.head()

In [ ]:
wiki_df.to_csv("travel_wiki_en_nlp1.csv")

### TF-IDF를 통한 문장요약&키워드 추출 및 WORD COUNT

http://excelsior-cjh.tistory.com/entry/TextRank%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EB%AC%B8%EC%84%9C%EC%9A%94%EC%95%BD

tf : 문장 내에서 특정 단어의 빈도수 

idf : 전체 문장의 수/ 특정 단어가 나오는 문장의 수

In [ ]:
df = pd.read_csv("travel_wiki_en_nlp1.csv", encoding = "cp949")
print(len(df))
df.head()

In [ ]:
df.iloc[0]

In [ ]:
result = []

for idx in range(len(df)):
    temp = df.iloc[idx]
    result.append((temp[0],(temp[1], temp[2], temp[3], temp[4], temp[5], temp[6], temp[7], temp[8])))

In [ ]:
result[0]

In [ ]:
class SentenceTokenizer() :
    def __init__(self, text) :
        self.text = text
        self.sentence = []
        self.noun = []
        self.word_count = []
        
    def making_sentence(self) :
        self.sentence = nltk.sent_tokenize(self.text)
        self.sentence = list(filter(lambda x: "=" not in x, self.sentence))
    
    def making_noun(self) :
        for sentence in self.sentence :
            if sentence != " " :
                tokens = nltk.word_tokenize(sentence)
                tagged = nltk.pos_tag(tokens)
                nn_tagged = list(filter(lambda x: "NN" in x[1], tagged))
                nn_tagged = list(map(lambda x: x[0], nn_tagged))
                self.noun.append(" ".join(nn_tagged))
                
    def word_counter(self) :
        dic = {}
        
        for noun in self.noun :
            noun_lst = noun.split()
            
            for n in noun_lst :
                if n in dic :
                    dic[n] += 1
                else :
                    dic[n] = 1
        
        self.word_count = sorted(dic.items(), reverse = True, key = lambda x : x[1])
        
    def get_sentence(self) :
        return self.sentence

    def get_noun(self) :
        return self.noun
    
    def get_word_count(self) :
        return self.word_count
    

In [ ]:
class GraphMatrix() :
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}



In [ ]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            
            if link_sum != 0:
                A[:, id] /= link_sum
                A[:, id] *= -d
                A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        
        new_A = A+0.00001*np.random.rand(len(A), len(A)) # noise 발생
        ranks = np.linalg.solve(new_A, B) # 연립방정식 Ax = b
        
        return {idx: r[0] for idx, r in enumerate(ranks)}



In [ ]:
class TextRank(object):
    def __init__(self, text):
        self.text = SentenceTokenizer(text)
        self.text.making_sentence()
        self.text.making_noun()
        self.text.word_counter()
            
        self.sentences = self.text.get_sentence()
        self.nouns = self.text.get_noun()
        self.counts = self.text.get_word_count()
        
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        
        if sent_num > len(self.sorted_sent_rank_idx) :
            sent_num = len(self.sorted_sent_rank_idx)
        
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        
        for idx in index:
            summary.append(self.sentences[idx])
            
        return summary
    
    def keywords(self, word_num=10):
        
        if word_num > len(self.sorted_word_rank_idx) :
            word_num = len(self.sorted_word_rank_idx)
        
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords



In [ ]:
err_lst = []

for idx, place in enumerate(result) :
    if place[1][6] in ("", " ")  : 
        continue
    
    try :
        if place[1][7] not in ("", " ") :
            f_textrank = TextRank(place[1][6])
            s_textrank = TextRank(place[1][7])   

            f_summary = f_textrank.summarize()
            s_summary = s_textrank.summarize()
            f_keyword = f_textrank.keywords()
            s_keyword = s_textrank.keywords()

            result[idx] = (place[0], (place[1][0], place[1][1], place[1][2], place[1][3], place[1][4], place[1][5], place[1][6], place[1][7], f_summary, s_summary, f_keyword, s_keyword, f_textrank.counts, s_textrank.counts))

        else :

            f_textrank = TextRank(place[1][6])
            s_textrank = []

            f_summary = f_textrank.summarize()
            s_summary = []
            f_keyword = f_textrank.keywords()
            s_keyword = []

            result[idx] = (place[0], (place[1][0], place[1][1], place[1][2], place[1][3], place[1][4], place[1][5], place[1][6], place[1][7], [] ,[], [],[], 0,0))
    except :
        err_lst.append((place[0], (place[1][0], place[1][1])))
        print(place[0],"err")
        result[idx] = (place[0], (place[1][0], place[1][1], place[1][2], place[1][3], place[1][4], place[1][5], place[1][6], place[1][7], [] ,[], [],[], 0,0))
    

In [ ]:
wiki_tfidf = pd.DataFrame.from_items(result).T
wiki_tfidf.head()

In [ ]:
wiki_tfidf.to_csv("travel_wiki_tfidf1.csv")